# HW6 (20')

<font size='4'>
    
Please submit your assignment as an HTML or PDF file.

Print your name (First and Last) below.

In [15]:
# Write your answer here:
Sapna David

SyntaxError: invalid syntax (3194835289.py, line 2)

<font size='4'>
    
Import the `pandas`, `matplotlib.pyplot`, `numpy`, `scipy` libraries and assign them with proper nicknames.

In [1]:
# Write your answer here:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import os
import openpyxl


In [7]:
# Do not delete this code section.
np.random.seed(100)

## 1. Write a function that output marginal summary statistics and missing values for continuous variable (10')

<font size='4'>

**Write the function with `def`** (6')
- Given a vector of continuous measure, you are asked to write a function named `fn_marginal_continuous`.
- The function has one parameter `input_vec` and outputs a list of summary measure and the number of missing values.
- Check the missingness of the `input_vec` using `np.isnan()` or `pd.isna()` functions.
    - In our first case, we will assume the `input_vec` is an numpy array with missing values marked with `np.nan`.
- The list of summary measure can be either **(mean, std)** or **(median, q1, q3)** depending on the normality assumption.
    - To determine the normality assumption, you can rely on the p-value of the Shapiro-Wilk test.
    - Relevant functions can be found here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html 
    - If the p-value < 0.05, it is not normally distributed, otherwise, you can treat it as normally distributed.
    - Think about what measure to report based on the normality assumption.
    - Part of relevant functions can be found here: https://numpy.org/doc/2.0/reference/generated/numpy.nanmean.html
- The return statement should include two components: `missing_num` and `output_ls` (your summmary measure).
    - The missing numbers should be of type `int` instead of `np.int64`, and summary values should be of type `float` instead of `np.float64`.
    - Round all your summary values such that they have no more than **3** digits after the decimals. 

In [28]:
# Write your defined function in this code chunk only.
from scipy import stats

def fn_marginal_continuous(input_vec):
    # Count missing values
    missing_num = int(np.sum(np.isnan(input_vec)))

    # Remove missing values for calculations
    valid_data = input_vec[~np.isnan(input_vec)]

    # Conduct Shapiro-Wilk test for normality
    stat, p_value = stats.shapiro(valid_data)

    # Check if data is normal
    if p_value < 0.05:
        # Not normally distributed → use median, Q1, Q3
        median = float(np.round(np.nanmedian(valid_data), 3))
        q1 = float(np.round(np.nanpercentile(valid_data, 25), 3))
        q3 = float(np.round(np.nanpercentile(valid_data, 75), 3))
        output_ls = [median, q1, q3]
    else:
        # Normally distributed → use mean, standard deviation
        mean = float(np.round(np.nanmean(valid_data), 3))
        std = float(np.round(np.nanstd(valid_data), 3))
        output_ls = [mean, std]
    return missing_num, output_ls


<font size='4'>

**Test your function with the following different arguments:** (4') <br>
For each scenario, please export the results as `missing_num_x`, `output_ls_x` and print them out separately.
1. A standard normal random vector with a sample size of 100, named `input_vec_1`.
2. A Chi-squared random vector with a degree of freedom 1 and a sample size of 100, named `input_vec_2`.
3. Change the first element of `input_vec_1` as `np.nan` and create a new array named `input_vec_3`.
    - Note that to create a copy of an numpy array, use `np.copy()` first.
4. Change the last element of `input_vec_2` as `np.nan` and create a new array named `input_vec_4`.

In [29]:
# Write your own code for scenario 1:
input_vec_1 = np.random.normal(0, 1, 100)

missing_num_1, output_ls_1 = fn_marginal_continuous(input_vec_1)

print("Scenario 1:")
print("missing_num_1:", missing_num_1)
print("output_ls_1:", output_ls_1)
print()

Scenario 1:
missing_num_1: 0
output_ls_1: [-0.117, 1.06]



In [36]:
# Write your own code for scenario 2:
input_vec_2 = np.random.chisquare(1, 100)

missing_num_2, output_ls_2 = fn_marginal_continuous(input_vec_2)

print("Scenario 2:")
print("missing_num_2:", missing_num_2)
print("output_ls_2:", output_ls_2)
print()

Scenario 2:
missing_num_2: 0
output_ls_2: [0.285, 0.068, 1.285]



In [20]:
# Write your own code for scenario 3:
input_vec_3 = np.copy(input_vec_1)
input_vec_3[0] = np.nan

missing_num_3, output_ls_3 = fn_marginal_continuous(input_vec_3)

print("Scenario 3:")
print("missing_num_3:", missing_num_3)
print("output_ls_3:", output_ls_3)
print()

Scenario 3:
missing_num_3: 1
output_ls_3: [0.206, 1.078]



In [21]:
# Write your own code for scenario 4:
input_vec_4 = np.copy(input_vec_2)
input_vec_4[-1] = np.nan

missing_num_4, output_ls_4 = fn_marginal_continuous(input_vec_4)

print("Scenario 4:")
print("missing_num_4:", missing_num_4)
print("output_ls_4:", output_ls_4)
print()

Scenario 4:
missing_num_4: 1
output_ls_4: [0.505, 0.182, 1.269]



## 2. Write a function that output marginal summary statistics and missing values for categorical variable (5')

<font size='4'>

**Write the function with `def`**
- Given a column vector, you are asked to write a function named `fn_marginal_categorical`.
- The function has one parameter named `input_vec` and outputs a list of summary measure and the number of missing values.
- Check the missingness of the `input_vec` using `np.isnan()` or `pd.isna()` functions.
    - In our second case, we will assume the `input_vec` is a column from a pandas DataFrame with missing values marked with `np.nan`.
    - You can use both functions to identify missing values and yield the number of missingness.
    - Use `pd.Series.value_counts()` function to obtain the frequency and proportion of `input_vec`, denoted as `tab_count` and `tab_percent`, respectively.
    - Details can be found here: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
    - For proportion, please use percentage (0-100%). You can ignore % when reporting.
- The return statement should include two components: `missing_num` and `output_tab` (your summmary measure).
    - For your `output_tab`, combine the count and proportion together using `pd.concat()`. Your `output_tab` should have three columns: variable name, count, and proportion.
    - Details can be found here: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
    - The missing numbers should be of type `int` instead of `np.int64`, and percentgae values should be of type `float` instead of `np.float64`.
    - Round all your relevant summary values such that they have no more than **2** digits after the decimals.

In [30]:
# Write your defined function in this code chunk only.
def fn_marginal_categorical(input_vec):
    # Count missing values
    missing_num = int(pd.isna(input_vec).sum())

    # Remove missing values for tabulation
    valid_data = input_vec.dropna()

    # Frequency count
    tab_count = valid_data.value_counts(dropna=False)

    # Proportion in percentage
    tab_percent = valid_data.value_counts(normalize=True, dropna=False) * 100

    # Round proportions and convert to float
    tab_percent = tab_percent.round(2).astype(float)

    # Combine counts and percentages into a single DataFrame
    output_tab = pd.concat([tab_count, tab_percent], axis=1)
    output_tab.columns = ['count', 'proportion']

    # Reset index and rename first column as 'variable'
    output_tab = output_tab.reset_index()
    output_tab.columns = ['variable', 'count', 'proportion']

    return missing_num, output_tab

## 3. Test your written functions with a real dataset. (3')

<font size='4'>
    
**Test the summary function for the continuous measure** (1')
- Load the `PTSD dataset.xlsx` and name it as `ptsd_df`.
    - The dataset should be stored under `data` folder when you sync changes and fetch origins the GitHub repository.
- Use the column `pcl5month_score.baseline` as the input vector. This is a continuous measure.
- Extract the corresponding column and give it a name `pcl5month_base`.
- (*Optional*) You convert from a pandas.dataframe to an numpy array.

In [83]:
# import ptsd dataset in this code section only (no point for this part as you have done it a few times)
os.chdir("/Users/sapnadavid/Documents/GitHub/BIOS-584/BIOS-584")
print( os.getcwd())
current_dir = os.getcwd()

# Define the file path
file_path = "/Users/sapnadavid/Documents/GitHub/BIOS-584/BIOS-584/data/PTSD dataset.xlsx"

# Load the specific sheet
ptsd_df = pd.read_excel(file_path, sheet_name="main_dataset", engine="openpyxl")

# Check that it loaded properly
print("Columns in 'main_dataset':\n", ptsd_df.columns.tolist())







/Users/sapnadavid/Documents/GitHub/BIOS-584/BIOS-584
Columns in 'main_dataset':
 ['record_id', 'ptsdpresent_caps', 'caps_minuspcl', 'caps_minuspcl_code', 'caps_intake', 'pcl5_score_intake', 'pcl5month_score.baseline', 'pcl5week_score.completion', 'pcl5month_score.3_month_follow_up', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code', 'phq9_score_intake', 'phq9_score.baseline', 'age_iop', 'gender_code', 'race_code', 'ethnicity_code', 'sexualorient_code', 'education_code', 'employment_code', 'rank_code', 'branch_code', 'total_ptsd_sx_caps', 'caps_1', 'caps_1_sev', 'caps_2', 'caps_2_sev', 'caps_3', 'caps_3_sev', 'caps_4', 'caps_4_sev', 'caps_5', 'caps_5_sev', 'b_sx_caps', 'b_sev_caps', 'criterion_b', 'caps_6', 'caps_6_sev', 'caps_7', 'caps_7_sev', 'c_sx_caps', 'c_sev_caps', 'criterion_c', 'caps_8', 'caps_8_sev', 'caps_9', 'caps_9_sev', 'caps_10', 'caps_10_sev', 'caps_11', 'caps_11_sev', 'caps_12', 'caps_12_sev', 'caps_13', 'caps_13_sev', 'caps_14', 'caps_14_sev', 'd_sx_caps', 'd_s

In [84]:
# test continuous measure
pcl5month_base = ptsd_df["pcl5month_score.baseline"]

# Optional: Convert to a numpy array
pcl5month_base_array = np.array(pcl5month_base)

# Display basic info
print("PCL-5 Month Baseline Summary:")
print(pcl5month_base.describe())

PCL-5 Month Baseline Summary:
count    474.000000
mean      51.164557
std       14.544323
min        4.000000
25%       42.000000
50%       51.000000
75%       62.000000
max       80.000000
Name: pcl5month_score.baseline, dtype: float64


<font size='4'>

**Test the summary function for the categorical measure** (2')

- Use the column `mdd_code` as the input vector. This is a binary vector.
1. Extract the corresponding column and give it a name `mdd_code_vec`. Output your results as `missing_num_1` and `tab_1`. Print each element out (You will write `print()` twice).
2. Create a copy of `mdd_code_vec` and name it as `mdd_code_vec_2`. Change its first element to `NaN`.
    - Rerun the `fn_marginal_categorical()` with the new vector. Output your results as `missing_num_2` and `tab_2`. Print each element out (You will write `print()` twice).

In [85]:
# Write your own code for scenario 1 only:
mdd_code_vec = ptsd_df["mdd_code"]

missing_num_1, tab_1 = fn_marginal_categorical(mdd_code_vec)


print(missing_num_1)
print(tab_1)


0
   variable  count  proportion
0         0    340       70.39
1         1    143       29.61


In [86]:
# Write your own code for scenario 2 only:
mdd_code_vec_2 = mdd_code_vec.copy()
mdd_code_vec_2.iloc[0] = np.nan


missing_num_2, tab_2 = fn_marginal_categorical(mdd_code_vec_2)


print(missing_num_2)
print(tab_2)

1
   variable  count  proportion
0       0.0    339       70.33
1       1.0    143       29.67


## 4. Lambda functions and for loop (2')

<font size='4'>

- Create a ``` lambda ``` function that checks if column `pcl5_score_intake` is greater than 30, denoted as `fn_pcl5_ptsd_check`.
- Create a new list `pcl5_score_intake_ls` that shows `True` if `pcl5_score_intake`>30 and `False` otherwise using a for loop.
    - You can also try `map()` function to iterate the function over `pcl5_score_intake`.
    - Syntax is simple: `map(function_name, iterable, ...)`. 
- Print out the number of patients with `pcl5_score_intake` over 30 and mark them as **Clinically Significant for PTSD**.
    - Your output can be something like `XXX out of YYY patients (ZZZ%) were marked as clinically significant for PTSD.`
    - Round your percentage with no more than **2** decimals.

In [87]:
# Write your own code
fn_pcl5_ptsd_check = lambda x: x > 30

pcl5_score_intake = ptsd_df["pcl5_score_intake"]

# Initialize empty list
pcl5_score_intake_ls = []

# Loop through each value and apply the lambda function
for score in pcl5_score_intake:
    pcl5_score_intake_ls.append(fn_pcl5_ptsd_check(score))

# check first few elements
print(pcl5_score_intake_ls[:10])

pcl5_score_intake_ls_map = list(map(fn_pcl5_ptsd_check, pcl5_score_intake))

num_clinically_significant = sum(pcl5_score_intake_ls)  # True counts as 1
total_patients = len(pcl5_score_intake_ls)

# Percentage
percent_clinically_significant = round((num_clinically_significant / total_patients) * 100, 2)

# Print output
print(f"{num_clinically_significant} out of {total_patients} patients ({percent_clinically_significant}%) were marked as clinically significant for PTSD.")

[True, True, True, True, True, True, True, True, True, True]
451 out of 483 patients (93.37%) were marked as clinically significant for PTSD.


<font size = "5">